In [ ]:
%pip install -r requirements.txt

In [1]:
import progressbar


def progress_bar():
    return progressbar.ProgressBar(maxval=78, widgets=[
        ' [', progressbar.Timer(), '] ',
        progressbar.Bar(marker='0', left='[', right=']'),
        ' (', progressbar.ETA(), ') ',
    ])


In [2]:
# Get all links to the detail page from digilib.uns.ac.id
import requests
from bs4 import BeautifulSoup as bs
import time
import json
import concurrent.futures

links = []

abstracts_data = []

pages = 1812

MAX_THREADS = 8
def get_link(page):
  time.sleep(1)
  print(f'Getting link from page {page}')
  res = requests.get(f'https://digilib.uns.ac.id/dokumen/fakultas/7/Fak-KIP/{page}')
  html_page = bs(res.content, 'html.parser')
  document_cards = html_page.select(
      '#digilib-body > div > div > div.col-md-8 > div.mb-5')
  for card in document_cards:
    document_type = card.select(
        '.dokumen-search-body .detail div:nth-child(2)')[0].text.strip().lower()
    if(document_type == 'skripsi'):
      anchor = card.find('a')
      if 'https://digilib.uns.ac.id/dokumen/detail/' in anchor.get('href'):
          links.append(anchor['href'])

def get_detail_links():
  with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
      executor.map(get_link, range(1, pages + 1))
  with open('detail_links.json', 'w') as output:
    output.write(json.dumps(links, indent=4, sort_keys=True))

get_detail_links()


Getting link from page 1Getting link from page 2

Getting link from page 3
Getting link from page 4
Getting link from page 5
Getting link from page 6
Getting link from page 7
Getting link from page 8
Getting link from page 9
Getting link from page 10
Getting link from page 11
Getting link from page 12
Getting link from page 13
Getting link from page 14
Getting link from page 15
Getting link from page 16
Getting link from page 17
Getting link from page 18
Getting link from page 19
Getting link from page 20
Getting link from page 21
Getting link from page 22
Getting link from page 23
Getting link from page 24
Getting link from page 25
Getting link from page 26
Getting link from page 27
Getting link from page 28
Getting link from page 29
Getting link from page 30
Getting link from page 31
Getting link from page 32
Getting link from page 33
Getting link from page 34
Getting link from page 35
Getting link from page 36
Getting link from page 37
Getting link from page 38
Getting link from pag

KeyboardInterrupt: 

In [ ]:
# # get abstract and title from detail page
# def get_abstract_and_title(link):
  
#   res = requests.get(link)
#   html_page = bs(res.content, 'html.parser')
#   table = html_page.find('table', {'class': 'table table-responsive'})
#   table_trs = table.find_all('tr')
#   nim = table_trs[2].text.strip()
#   if 'K35' in nim or 'K.35' in nim or 'K. 35' in nim or 'K 35' in nim:
#     print('==============================')
#     print(
#         f'Getting abstract from link: {link} \n')
#     tdabstrak = table_trs[13].find_all('td')
#     tdjudul = table_trs[4].find_all('td')
#     abstract = tdabstrak[2].get_text()
#     title = tdjudul[2].get_text()

#     abstracts_data.append({
#         'title': title,
#         'abstract': abstract,
#     })
#   else:
#     print('==============================')
#     print(
#         f'Link skipped: {link} \n')
#     time.sleep(1)


# def add_id_to_abstract(i):
#   abstracts_data[i]['id'] = i + 1


# def get_abstracts():
#   with open('detail_links.json', 'r') as input:
#     links_from_json = json.load(input)
#     with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#         executor.map(get_abstract_and_title, (link for link in links_from_json))
#     with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
#         executor.map(add_id_to_abstract,
#                      (i for i in range(len(abstracts_data))))
#     with open('abstracts_data.json', 'w') as output:
#       output.write(json.dumps(abstracts_data, indent=4, sort_keys=True))


# get_abstracts()

In [ ]:
# case folding
import json
import re
import time
import sys
import concurrent.futures

with open('data/abstracts_data.json') as abstracts_json:
  abstracts = json.load(abstracts_json)

bar = progress_bar()
bar.start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  answer = re.sub('[^a-z]+', ' ', abstract['abstract'].casefold())
  abstracts[i]['abstract'] = answer

with open('data/case_folded.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))
  
bar.finish()
print('Case folding done')

In [ ]:
# translate if english
import json
from langdetect import detect
import translators as ts

with open('data/case_folded.json') as case_folded_json:
  abstracts = json.load(case_folded_json)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  detector = detect(abstract['abstract'])
  if detector == 'en':
    translation = ts.google(abstract['abstract'], from_language='en', to_language='id')
    abstracts[i]['abstract'] = translation


with open('data/translated.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))
bar.finish()

In [ ]:
# case folding again
import json
import re
import time
import sys
import concurrent.futures


processed = []
count = 0
with open('data/translated.json') as abstracts_json:
  abstracts = json.load(abstracts_json)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  answer = re.sub('[^a-z]+', ' ', abstract['abstract'].casefold())
  abstracts[i]['abstract'] = answer

with open('data/case_folded.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))

bar.finish()
print('Case folding done')


In [ ]:
# stemming
import json
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

with open('data/case_folded.json') as case_folded_json:
  abstracts = json.load(case_folded_json)

factory = StemmerFactory()
stemmer = factory.create_stemmer()


bar = progress_bar()
bar.start()

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  stemmed = stemmer.stem(abstract['abstract'])
  abstracts[i]['abstract'] = stemmed


with open('data/stemmed_abstracts.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))

bar.finish()

In [ ]:
# filtering
import json
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

with open('data/stemmed_abstracts.json') as json_file:
  abstracts = json.load(json_file)

bar = progress_bar().start()

additional_stopwords = []

stopwords = StopWordRemoverFactory().get_stop_words() + additional_stopwords

for i, abstract in enumerate(abstracts):
  bar.update(i + 1)
  words = abstract['abstract'].split()
  filtered = ''
  for word in words:
    if word not in stopwords:
      filtered += word + ' '
  abstracts[i]['abstract'] = filtered

with open('data/filtered.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))   

In [ ]:
import json
import re
import time
import sys
import nltk

nltk.download('punkt')

with open('data/filtered.json') as json_file:
  abstracts = json.load(json_file)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  frequency = nltk.FreqDist(nltk.word_tokenize(abstract['abstract']))
  abstracts[i]['frequency'] = frequency
  bar.update(i + 1)

with open('data/tokenized.json', 'w') as outfile:
  outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))

In [ ]:
# term frequency
import json
import nltk

with open('data/tokenized.json') as json_file:
  abstracts = json.load(json_file)

bar = progress_bar().start()

for i, abstract in enumerate(abstracts):
  result = {}
  bagOfWordsCount = len(abstract['abstract'])
  for word, count in abstract['frequency'].items():
    if(count / float(bagOfWordsCount)):
      result[word] = count / float(bagOfWordsCount)
  abstracts[i]['frequency'] = result
  bar.update(i + 1)

with open('data/termfrequency.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))


In [ ]:
# inverse document frequency
import json
import nltk
import math


threshold = 4

with open('data/tokenized.json') as json_file:
  abstracts = json.load(json_file)

bar = progress_bar().start()

abstract_length = len(abstracts)

for i, abstract in enumerate(abstracts):
  idfDict = dict.fromkeys(abstract['frequency'].keys(), 0)
  result = {}
  for key in idfDict.keys():
    for abstract in abstracts:
      if key in abstract['frequency']:
        idfDict[key] += 1
  
  for word, val in idfDict.items():
    if val > threshold:
      result[word] = math.log10(abstract_length / float(val) + 1)
  abstracts[i]['frequency'] = result
  bar.update(i + 1)

with open('data/idf.json', 'w') as outfile:
	outfile.write(json.dumps(abstracts, sort_keys=True, indent=4))


In [ ]:
# tfidf
import json

with open('data/idf.json') as idf_json:
  idfs = json.load(idf_json)

with open('data/termfrequency.json') as tf_json:
  tfs = json.load(tf_json)

bar = progress_bar().start()

for i, idf in enumerate(idfs):
  tfidf = {}
  maxtfidf = 0
  for word, val in idf['frequency'].items():
    if tfs[i]['frequency'].get(word) is not None:
      tfidf[word] = val*float(tfs[i]['frequency'].get(word))
      if maxtfidf < tfidf[word]:
        maxtfidf = tfidf[word]
        maxtfidfword = word
  idfs[i]['frequency'] = tfidf
  idfs[i]['maxtfidfword'] = maxtfidfword
  bar.update(i + 1)

with open('data/tfidf.json', 'w') as json_file:
  json_file.write(json.dumps(idfs, sort_keys=True, indent=4))



In [31]:
# creating dictionary
import json

with open('data/tfidf.json') as json_file:
  documents = json.load(json_file)

bar = progress_bar().start()
words = []

for i, document in enumerate(documents):
  bar.update(i + 1)
  for word, val in document['frequency'].items():
    if word not in words:
      words.append(word)

words_dict = dict(zip(range(len(words)), words))

with open('data/dictionary.json','w') as outfile:
	outfile.write(json.dumps(words_dict, sort_keys=True, indent=4))




In [32]:
import json

with open('data/dictionary.json') as json_file:
  words = json.load(json_file)
with open('data/tfidf.json') as json_file:
  documents = json.load(json_file)

# define function to get key from dictionary
def get_key(val):
  for key, value in words.items():
    if val == value:
      return key

for i, document in enumerate(documents):
  bar.update(i + 1)
  result = {}
  for word, val in document['frequency'].items():
      if word in words.values():
        result[get_key(word)] = val
  documents[i]['frequency'] = result

with open('data/dictionarized.json','w') as outfile:
	outfile.write(json.dumps(documents, sort_keys=True, indent=4))


In [29]:
# hierarchical clustering
import json
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import scipy.cluster.vq as scv
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from pandas import DataFrame
from sklearn.neighbors import NearestCentroid
from sklearn.manifold import MDS

N_CLUSTERS = 10

with open('data/dictionary.json') as json_file:
  dictionary = json.load(json_file)

with open('data/dictionarized.json') as json_file:
  documents = json.load(json_file)

bar = progress_bar().start()

print('Calculating Clusters')
array = []
for i, document in enumerate(documents):
	bar.update(i + 1)
	new = []
	for key in dictionary.keys():
		if not document['frequency'].get(key) is None:
			new.append(document['frequency'].get(key))
		else:
			new.append(0)
	array.append(new)

X = np.array(array)

titles = []

for document in documents:
	titles.append(document['title'])

clustering = AgglomerativeClustering(n_clusters=N_CLUSTERS, affinity='euclidean', linkage='ward')
cluster = clustering.fit_predict(X)

# for i in range(len(cluster)):
# 	print(f'Cluster: {cluster[i]}, Title: {titles[i]} \n')

# fig, dendogram = plt.subplots(figsize=(30,40))

# dendogram = sch.dendrogram(sch.linkage(array, method='ward'), orientation='top')
# plt.title('Hierarchical Clustering Dendrogram')
# plt.ylabel('Article')
# plt.xlabel('Euclidean Distance')
# plt.tight_layout()
# plt.savefig('t'+str(4)+'sw'
#             +'dendogram.png', dpi=200)

dataframe = DataFrame({
		'Cluster': cluster,
		'Title': titles,
})

sorted_data_frame = dataframe.sort_values(by=['Cluster'])
sorted_data_frame.to_csv('hierarchical_clustering.csv', index=False)


Calculating Clusters
